R for maps

Libraries

In [ ]:
library(tidyverse)
library(sf)
library(tmap)
library(spData)

Example: US Counties, Georgia

Major US Roads: 
https://catalog.data.gov/dataset/tiger-line-shapefile-2016-nation-u-s-primary-roads-national-shapefile
GA counties: 
https://arc-garc.opendata.arcgis.com/datasets/dc20713282734a73abe990995de40497_68


Read data, filter by name

In [ ]:
data("us_states") # from spData package
ga <- us_states %% filter(NAME == "Georgia")
counties <- st_read("myfolder/counties.shp", quiet=T)
roads <- st_read("myfolder/roads.shp")

Create projection variable

In [ ]:
proj <- "+proj=lcc +lat_1=31.41666666666667 +lat_2=34.28333333333333 +lat_0=0 +lon_0=-83.5 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs"

Reproject inputs

In [ ]:
roads <- st_transform(roads, proj)
ga <- st_transform(ga, proj)
counties <- st_transform(counties, proj)

Clip road by selected state input

In [ ]:
roads <- st_crop(roads, ga)

Plot state, counties, and roads

In [ ]:
plot(st_geometry(ga))
plot(st_geometry(counties), add = T, col = "red")
plot(st_geometry(roads), add = T, col = "blue")

------------

Using ggplot

In [ ]:
ggplot(ga) +  geom_sf()

with title

In [ ]:
ggplot(ga) + 
  geom_sf(fill = "beige") +  
  labs(title = "The fine state of Georgia") +  
  theme_minimal()

with more info

In [ ]:
ggplot() +
  geom_sf(data = counties, aes(fill = totpop10)) +
  geom_sf(data = roads, color = "orange") +
  labs(title = "All roads lead to the ATL", fill = "Population") +
  theme_minimal()

Using ggspatial, north arrow, scalebar

In [ ]:
library(ggspatial)
ggplot() +
  geom_sf(data = counties, aes(fill = totpop10)) +
  geom_sf(data = roads, color = "orange") +
  labs(title = "All roads lead to the ATL", fill = "Population") +
  theme_minimal() +
  annotation_scale(location = "bl", width_hint = 0.5) +
  annotation_north_arrow(location = "tr", which_north = "true", style = north_arrow_fancy_orienteering())

Using tmap

In [ ]:
library(tmap)
tmap_mode("plot") 
qtm(roads)

choropleth quick thematic map

In [ ]:
qtm(counties, fill = "totpop10", fill.title = "Total population")

using tm shape

In [ ]:
tm_shape(counties) +
    tm_fill(col = "totpop10", convert2density = T, style = "jenks", 
    title = "Population (x100)") +
    tm_borders(alpha = 0.3) +
    tm_compass() +
    tm_scale_bar()

Save map

In [ ]:
save_tmap(tm, "./fig/my_awesome_map.png", width = 800, height = 1000)

View interactive map

In [ ]:
tmap_mode("view")
tm